In [1]:
# Βιβλιοθήκες
import pandas as pd
import numpy as np
import gekko as gk
import seaborn as sns
import xarray as xr
from matplotlib import pyplot as plt
import typing
import itertools
import math
import functools
from pca import pca
from IPython.display import display, HTML
import prince
import networkx
import sklearn
import tensorflow as tf
from tensorflow import keras
import multiprocessing
from imblearn.over_sampling import ADASYN, SMOTE
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product
import arviz as az
import aesara
from factor_analyzer import FactorAnalyzer
import aesara.tensor as at
import scipy
import pymc as pm, pymc
import gpflow
from pymc.sampling.jax import sample_numpyro_nuts
import numpyro
import pickle

2022-12-10 11:50:27.008595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:50:27.008994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:50:27.009006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/media/alexander-fyrogenis/Elements/Διδακτορικό/Olive Oil/notebooks/evoos_env/lib/python3.10/

In [2]:
# Γενικές επιλογές εμφάνισης
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',30)

numpyro.set_platform("gpu")

figsize=(13,6)
fontsize=10
sns.set(rc={'figure.figsize':(15,10),
           'font.size':5})

In [3]:
# Utilities
from sklearn.preprocessing import StandardScaler

std_scale = lambda df: pd.DataFrame(data = StandardScaler().fit_transform(df), columns = df.columns, index=df.index)

def render_df(df:pd.DataFrame):
    from IPython.display import display, HTML
    display(HTML(df.to_html()))
    return

def full_display(r=None, c=None):
    pd.set_option('display.max_rows', r)
    pd.set_option('display.max_columns',c)
    return

def reset_display():
    pd.set_option('display.max_rows',10)
    pd.set_option('display.max_columns',30)

def full_display_once(df:pd.DataFrame, r=None, c=None):
    full_display(r=r, c=c)
    render_df(df)
    reset_display()

def rowwise_value_counts(df:pd.DataFrame):
    vcounts_df = pd.DataFrame(data = df.apply(lambda x: x.value_counts()).T.stack()).astype(int).T
    vcounts_df.index = ['']
    return vcounts_df

invert_dict = lambda e: {v:k for k, v in e.items()}

def categorical_scatter(X:pd.DataFrame, Y:typing.Optional[pd.DataFrame],cols:int=3,max_rows:int = 3,
                             figsize:tuple[int, int] = (30, 15), xaxis_label_size: int=12,
                             yaxis_label_size:int=12, categorical:typing.Optional[str]='hue')->typing.Optional[plt.figure]:
    '''
        Generate pair-wise scatter plots for a given DataFrame, with optional support for large Datasets
        and categorical variables. Yields a figure with `max_rows x cols` scatterplots per call. When
        `categorical=None` only generates pair-wise scatterplots for the columns of X, else generates all
        combinations of X-column pair and Y columns. When `categorical='hue'` values of the categorical Y
        variable will be depicted color-coded and when `categorical=size` different factors of the Y variable
        will have different sizes instead.
        
        Args:
        -----
        
            - X:pandas.DataFrame := The data to depict. When Y is also specified, X is assumed to be the DataFrame
            of indicator variables
            
            - Y:Optional[pandas.DataFrame] := Optional Dataframe of categorical variables to depict. Ignored if 
            `category` is `None`.
            
            - idx_lvl:int=1 := For multilevel indexed dataframes the lowest level to squash the index on. Must be non-negative
            
            - cols:int=3 := Number of columns for the resulting facet grid plot. Must be non-negative. Defaults to 3
            
            - max_rows:int=3 := Maximum number of rows per batch the generator yields. Defaults to 3 and must be non-negative
            
            - figsize:tuple[int, int] := A `height x width` tuple for the generated plots. Defaults to `(30, 15)`
            
            - xaxis_label_size:int=12 := The size of the x-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - yaxis_label_size:int=12 := The size of the y-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - categorical:Optional[str] := Set the display of the categorical variable. (1) `None` ignores `Y` and only displays X
            pair-wise scatterplots, (2) 'hue' displays the categorical variable of `Y` as color and (3) 'size' displays
            the categorical variable with differently sized points
            
        Returns:
        --------
        
            - fig:matplotlib.pyplot.figure := The figure object, a FacetPlot of scatterplots
            
       
       Raises:
        ------
        
            - WIP
            
            - ValueError 
    '''
    XY = pd.concat([Y, X], axis=1)
    idx_lvl= XY.columns.nlevels-1
    skip_multiindex = lambda df,i , idx_level=idx_lvl: df.columns[i][idx_level] if idx_level else df.columns[i]
    if idx_lvl:
        XY.columns = XY.columns.get_level_values(idx_lvl)
    x_combs = math.comb(X.shape[1], 2)
    MAX_ROWS = max_rows
    
    if categorical is not None:
        plots = x_combs*Y.shape[1]
    else:
        plots = x_combs
    ncols = cols
    size_scaling_factor = None
    total_rows = math.ceil(plots/ncols)
    total_figures = math.ceil(total_rows/MAX_ROWS)
    X_pairs = itertools.combinations(range(X.shape[1]), 2)
    subplots = itertools.product(X_pairs,range(Y.shape[1]) ) if categorical is not None else X_pairs
    exhaustion_sentinel = False
    while True:
        if exhaustion_sentinel: break
        fig, axs = plt.subplots(nrows=MAX_ROWS, ncols=ncols, figsize=figsize)
        ax_indices = itertools.product(range(MAX_ROWS), range(ncols), repeat=1)
        plotslice = itertools.islice(subplots, MAX_ROWS*ncols)
        fig_generator = itertools.zip_longest( ax_indices, plotslice, fillvalue = ((None, None), None) )
        for (axi, axj), e in fig_generator:
            e = tuple(flatten(e))
            if e[0] is not None:
                if categorical == 'size':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       size=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical == 'hue':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       hue=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical is None:
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       ax=axs[axi, axj], legend=True)
                axs[axi, axj].set_xlabel(skip_multiindex(X, e[0]))
                axs[axi, axj].set_ylabel(skip_multiindex(X, e[1]))
                axs[axi, axj].xaxis.label.set_size(xaxis_label_size)
                axs[axi, axj].yaxis.label.set_size(yaxis_label_size)
                
            else:
                axs[axi, axj].axis('off')
                exhaustion_sentinel = True
        yield fig 
    return None



def flatten(xs):
    from collections.abc import Iterable
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

def tidy_multiindex(df:pd.DataFrame, sep:str="."):
    '''
        Compress a hierarchically indexed dataframe to standardized tidy
        format. A unique sepperator `sep` is used to allow reversal. All
        levels of the index are appended together with a delimeter to allow
        reversals.
        
        Args:
        ----
        
            - df:pandas.DataFrame := A `pandas.DataFrame` hierarchically indexed
            
            - sep:str='_._' := A delimenter delineating the different levels
            of the index. Ensure it is not present in any column name to avoid
            a malformed index
            
        Returns:
        --------
        
            - ndf:pandas.DataFrame := The DataFrame with a single-level index
    '''
    tidy_cols = (functools.reduce(lambda e1,e2: str(e1)+sep+str(e2), col ) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = tidy_cols
    return ndf

def reverse_tidy_multiindex(df:pd.DataFrame, sep="."):
    '''
        Reverses the tidying to a hierachical format. Different
        levels of the index are identified based on "sep"
        
        Args:
        -----
        
            - df:pandas.DataFrame := The dataframe to process
            
            - sep:str='_._' := The string delimeter, sepperating
            values for different levels of the index
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The dataframe with hierarchical index
    '''
    h_cols = (tuple(col.split(sep)) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = pd.MultiIndex.from_tuples(h_cols)
    return ndf

def undummify(df:pd.DataFrame,cols:list[str, tuple[str]],ncol_name:typing.Union[str,tuple[str]],
              sep:typing.Optional[str]=None,
              rmap:typing.Optional[dict[int, typing.Union[str, tuple[str]]]]=None
             )->pd.DataFrame:
    '''
        Reverses hot-encoded variables in the DataFrame. A series of hot-encoded
        variable levels $(i_1, i2, \dots, i_k)$ is mapped to a single new column
        $(k)$, whose name is specified by `ncol_name`, in the new dataframe. Pre
        vious level columns are dropped.
        
        Args:
        ----
        
            - df:pandas.DataFrame := The DataFrame to operate upon
            
            - cols:list[str, tuple[str]] := A list of columns, representing the
            levels of a categorical variable
            
            - sep:Optional[str] := sepperator for variable level. Currently ignored
            
            - ncol_name:Union[str, tuple[str]] := Name of the new categorical column
            
            - remap:Optional[dict[int, Union[str, tuple[str]]]] := A dictionary mapping
            of categorical levels to values. Keys are the assumed to be levels, values
            are assumed to be values (i.e. strings). When provided, the previous levels
            will be replaced by the specified mappings in the new DataFrame
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The processed dataframe
     '''
    _df = coredf.loc[:, cols]
    for i, col in enumerate(cols, 1):
        _df.loc[:, col] = i*_df.loc[:, col]
    ndf = df.copy(deep=True)
    ndf.drop(cols, axis=1, inplace=True)
    ndf[ncol_name] = _df.max(axis=1)
    c1 = df.columns.tolist()
    i = c1.index(cols[0])
    swp = ndf.columns.tolist()[:i-1]+[ndf.columns.tolist()[-1]]+ndf.columns.tolist()[i:-1]
    ndf = ndf.loc[:, swp]
    if rmap is not None:
        ndf = ndf.replace(rmap)
    return ndf
list_difference = lambda l1, l2: [e for e  in l1 if e not in set(l2)]

def corrspace_graph(df:pd.DataFrame):
    '''
        Generate a correlation graph representation of `df` datasets.
        Every node in the resulting graph is a variable and every 
        vertex between two nodes represents the correlation between
        the two variables, the correlation being the verted weight
        
        Args:
        ----
        
            - df:pandas.DataFrame := A dataset to depict
            
        Returns:
            - WIP
    '''
    
    return 


class DictTable(dict):
    '''
        Jupyter utility class that overrides the dicts' defaults
        __repr__ rendering the input dictionary to an HTML table
        for convenient jupyter rendering
    '''
    def _repr_html_(self):
        html = ["<table>"]
        for key, value in self.items():
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{0}</td>".format(value))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
    
def boxplots(df:pd.DataFrame, max_features:int=20, title:str="Boxplot",
             **kwargs)->typing.Generator[plt.Figure, None, None]:
    '''
        Wrapper around `pandas.boxplot` for datasets with a large number
        of features. Returns a generator of boxplots of subsets of the total
        features.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The DataFrame to plot
            
            - max_features:int>0 := The maximum number of features to plot
            in each generated boxplot
            
            - title:str='Boxplot' := The title of the plot. A subplot index
            of the form **(X/Y)**, tracking the current plot will be
            appended prior to rendering
            
            - **kwargs:dict[str:Any] := Keyword arguments to be be forwarded
            to pandas.DataFrame.boxplot
            
        Returns:
        -------
        
            - Generator[plt.Figure, None, None] := A generator yielding boxplots
            of feature subsets
    '''
    nplots = math.ceil(df.shape[1]/max_features)
    for i in range(nplots):
        if (i+1)*max_features<df.shape[1]:
            ndf = df.iloc[:,i*max_features:(i+1)*max_features]
        else:
            ndf = df.iloc[:,i*max_features:df.shape[1]]
        fig = ndf.boxplot(**kwargs)
        fig.set_title(title + " ({current}/{total})".format(current=i+1, total=nplots) )
        yield fig
        
def count_missing_nan(df:pd.DataFrame, axis:int=0):
    '''
        Return a new DataFrame with the counts of missing
        and invalid values across specified axis.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The data
            
            - axis:int=0 := The axis across which missing
            values will be enumerated. Defaults to 0 (show
            missing values in each column)
            
        Returns:
        -------
        
            - missing_df:pd.DataFrame := A DataFrame containing
            counts of missing values
    '''
    ndf = pd.DataFrame(df.isna().sum(axis=axis)).T
    ndf.index = ['']
    return ndf


class PlotPrior:
    '''
        WIP
    '''
    def __init__(self,data, columns:typing.Optional[list[str, tuple[str]]]=None,
                 visible:bool=True, stack:bool=True, show_grid:bool=False, ncols:int=3,
                kde:bool=True):
        self.columns=columns
        self.visible=visible
        self.stack=stack
        self.ncols=ncols
        self.kde=kde
        self.plots={} 
    
    def __prep_data(self, data):
        stacked = data.prior.stack(samples=["chain", "draw"])
        if self.columns is None:
            self.columns = data.columns
        for col in self.columns:
            self.plots[col] = pd.DataFrame(data=stacked[col].values.T)
    
    def __call__(self, data, var:str):
        df = self.plots[var]
        k = df.shape[1]
        n = self.ncols
        m = (k - 1) // n + 1
        fig, axes = plt.subplots(nrows=m, ncols=n, figsize=(n * 5, m * 3))
        for i, (name, col) in enumerate(self.plots[var].items()):
            r, c = i // n, i % n
            ax = axes[r, c]
            col.hist(ax=ax)
            if self.kde:
                ax2 = col.plot.kde(ax=ax, secondary_y=True, title=name)
                ax2.set_ylim(0)
        fig.tight_layout()
        return fig,axs
    
def plot_kernel():
    '''
        Utility method for visualizing GP kernels. Produces a square visualization
        of the selected kernel and all possible combinations of hyperparameters
        
        TODO
        +++++
        
            - Extend this for combination kernels by nesting input hyperparameter
            dicts
            
            - Raise combinatronic explsion error when too many hyperparameter values
            are specified
        
        Args:
        ----
        
            - 
            
        Returns:
        --------
        
            -fig:matplotlib.pyplot.Figure := The generated figure
    '''
    
class MultiLayerPerceptronKernel(pymc.gp.cov.Covariance):
    '''
        Multi layer Perceptron Kernel a.k.a Arcsine Kernel
        Code transplanted from `GPy.kern.src.mlp`. This kernel
        is designed to mimic the highly nonlinear processes 
        of a Neural Network and can be thought of as the limit
        of an A.N.N. with a single input layer as the number
        of neurons in the hidden layer goes to infinity. Then
        the N.N., with normal priors over the weights and biases
        becomes equivalent to a G.P. This Kernel is given by:
        
        .. math::

          k(x,y) = \\sigma^{2}\\frac{2}{\\pi }  \\text{asin} \\left
          ( \\frac{ \\sigma_w^2 x^\\top y+\\sigma_b^2}{\\sqrt{\\sigma_w^2x^\\top x +
          \\sigma_b^2 + 1}\\sqrt{\\sigma_w^2 y^\\top y + \\sigma_b^2 +1}} \\right )
          
        Args:
        -----

            - input_dims:int=1 := The number of input dimentions (columns) used for Covariance
            computations. Defaults to 1 (use the first column only).

            - active_dims:Optional[list[int]]=None := None or array-like specifying which input
            dimentions will be used in computing the covariance matrix. Can be specified as a
            a boolean vector or a vector of indices. Optional. Defaults to None, and the first
            `input_dims` columns are selected.

            - variance:Optional[float] := The output kernel variance scaling parameter, usually
            denoted `\eta`. Defaults to 1.0. Must be positive.

            - weight_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance
            of the weight in the A.N.N. Can be specified as either a scalar or a matrix of
            appropriate size (same as the first input matrix). Optional.  Defaults to 1.0.

            - bias_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance of
            the biases in the A.N.N. Can be either a scalar or a matrix of appropriate size. When
            computing covariance matrices (single input) must be a N-length vector of biases for a
            `N \times M` input matrix. When computing cross-covariance matrices, must be an
            `N \times N` matrix. Optional.  Defaults to 1.0.

            - ARD:bool=False := A(utomatic)R(elevance)D(etermination) flag. Unused and raises
            an error if switched. Optional. Defaults to False.
        
        
        Returns:
        -------
        
            - K:np.typing.NDArray := When called with the `.full(X)` or the `.full(X, Y)` method,
            computes and returns the covariance or cross-covariance matrices.
            
            - diag(K):typing.NDArray := When called with the `.diag(X)` or `.diag(X, Y)` method
            computes and returns the diagonal of the covariance or cross-covariance matrices
    '''
    four_over_tau = 2./np.pi
    
    def __init__(self,input_dims:int, active_dims:typing.Union[list[int], int]=None,
                 variance:float=1.0,
                 weight_variance:typing.Union[float, np.typing.NDArray]=1.0,
                 bias_variance:typing.Union[float, np.typing.NDArray]=1.0, ARD:bool=False):
        super(MultiLayerPerceptronKernel, self).__init__(input_dims, active_dims=active_dims)
        self.variance = variance
        self.weight_variance = weight_variance
        self.bias_variance = bias_variance
        if ARD:
            raise NotImplementedError("Automatic Relevance Determination, not implemented")
        self.ARD = ARD

    
    def _comp_prod(self, X, X2=None):
        if X2 is None:
            return (at.math.square(X)*self.weight_variance).sum(axis=1)+self.bias_variance
        else:
            return (X*self.weight_variance).dot(X2.T)+self.bias_variance
    
    def diag(self, X):
        """Compute the diagonal of the covariance matrix for X."""
        X_prod = self._comp_prod(X)
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(X_prod/(X_prod+1.))
    
    def full(self, X, X2=None):
        if X2 is None:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = X_denom
            X2 = X
        else:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = at.math.sqrt(self._comp_prod(X2)+1.)
        XTX = self._comp_prod(X,X2)/X_denom[:,None]/X2_denom[None,:]
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(XTX)
    
def advi_inspect_ELBO(fit_data, show:bool=True):
    '''
        Utility method to investigate ADVI fit.
        Returns log-ELBO against iterations.
    '''
    _h_advi_hist = fit_data.hist
    advi_elbo = pd.DataFrame(
        {'$log-ELBO$': -np.log(_h_advi_hist),
         'n': np.arange(_h_advi_hist.shape[0])})
    fig = sns.lineplot(y='$log-ELBO$', x='n', data=advi_elbo)
    if show:
        plt.show()
    return fig

def select_subarray(df:pd.DataFrame, targets:list[tuple[str,str,str]], indicators:list[tuple[str,str,str]],
                   scaleX:bool=True, train_split:bool=False,
                    mappings:typing.Optional[dict[str, typing.Union[str,int, float]]]=None,
                    dummify:bool=False,
                   )->tuple[pd.DataFrame, pd.DataFrame]:
    X = df.loc[:,indicators]
    X = std_scale(X) if scaleX else X
    Y = df.loc[:, targets]
    m = Y.loc[~Y.isna().any(axis=1),:].index.intersection(X.loc[~X.isna().any(axis=1),:].index)
    X = X.loc[m, :]
    Y = Y.loc[m,:]
    Y=Y.replace(mappings) if mappings is not None else Y
    Y = pd.get_dummies(Y) if dummify else Y
    if train_split:
        X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y,test_size=.1 ,random_state=44)
        return X_train, X_test, Y_train, Y_test
    else:
        return X, Y

In [4]:
def tf_model(Xtrain, Xtest, Ytrain, Ytest, loss:str='categorical_crossentropy',
             optimizer:str='adadelta',
             layers:typing.Optional[list[tuple[int, str]]]=None,
             metrics=['accuracy'], summary:bool=True, epochs:int=10,
             factivation:str='sigmoid',batch:int=64, callbacks:list=[],
            verbosity:int=1):
    M = Xtrain.shape[1]
    input_layer = tf.keras.Input(shape=(M,))
    x = input_layer
    for neurons, activation in layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)

    x = tf.keras.layers.Dense(math.ceil(Ytrain.shape[1]), activation=factivation)(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer=optimizer, loss=loss, 
                metrics=metrics)
    if summary:
        model.summary()
    model.fit(Xtrain, Ytrain, epochs=epochs, validation_data = (Xtest, Ytest),
              batch_size=batch, callbacks=callbacks, verbose=verbosity)
    return model
# Multiple query utility
query_multiple = lambda df, col, multiquery : functools.reduce(lambda p,n: "{col} == '{p}' | ".format(p=p, col=col) + "{col} == '{n}".format(n=n, col=col), multiquery).split(" == '",1)[1]+"'"

In [3]:
def load_data():
    with open('core_mappings.pickle', 'rb') as handle:
        core_mappings = pickle.load(handle)
    with open('r_core_mappings.pickle', 'rb') as handle:
        r_core_mappings = pickle.load(handle)
    df = pd.read_excel('evoos_processed.xlsx',header=2)
    return core_mappings,r_core_mappings,df
core_mappings, r_core_mappings, df = load_data()

#### Διερεύνηση Διαφοροποίησης Χημικού Προφίλ Βάση Ωρίμανσης

In [ ]:
fig, ax = plt.subplots(figsize=(25,9))
sns.heatmap(pd.concat([Y,X], axis=1).corr(method='kendall').iloc[Y.shape[1]:,:Y.shape[1]].applymap(np.abs),
            cmap='viridis', ax=ax, annot=True)
ax.set_title("Kendal Correlations Between Chemical Predictors and Continuous Production Targets")
plt.show()

Αφού υπάρχει το επίμονο πρόβλημα οτι το Ν.Δ. προβλέπει τις παραμέτρους με τις μέσες τιμές (δηλαδή δεν αξιοποιεί κάποια χημική πληροφορία αλλά μαντεύει με τη μέση ή διάμεση) εξαιτάζουμε τις συσχετίσεις Kendall μεταξύ των παραμέτρων ενδιαφέροντως. Στη περιοχή των συνεχών μεταβλητών, παρατηρούμε κάποιες ασθενείς αλλά υπαρχείς συσχετίσεις. Για παράδειγμα υπάρχει μια συσχέτιση θερμοκρασίας μάλαξης και συγκέντρωσης λινολεϊκού $\approxeq 20\%$, ασθενής αλλά μη-αμεληταία. Η μέση ακρίβεια μας για την ποικιλία, είναι της τάξεως του $\approxeq 70\%$ και χρειάζεται βελτίωση. Παρατηρούμε οτι στο πίνακα συσχετίσεων Kendall φαίνεται να υπάρχει διαφοροποίηση μόνο στη περίπτωση "ΠΑ" και "Μ" ωρίμανσης, ενώ για τις περιπτώσεις "ΠΙ" και "Π" πρακτικά καμία συσχέτιση με το χημικό προφίλ. Ενδέχεται η παρουσία αυτών των κατηγοριών να μειώνει την ακρίβεια

Αρκετές διαδικασίες στατικής συμπερασματολογίας αφορούν τη σύγκριση μεταξύ ομάδων. Συχνά ενδιαφερόμαστε για το αν δύο σύνολα, που αντιπροσωπεύουν ομάδες είναι διαφορετικά μεταξύ τους ή το ένα μεγαλύτερο από το άλλο. Χρειάζεται κατάλληλο στατιστικό μοντέλο για να απαντηθούν τέτοιου είδους ερωτήματα γιατί οι όποιες διαφορές συνοδεύονται συνήθως από στοχαστικό θόρυβο μπορεί να οδηγήσει σε παραπλανητικά συμπεράσματα για τις παρατηρούμενες διαφορές.

Η *de facto* μέθοδος αντιμετόπισης τέτοιων ερωτημάτων εμπλέκει κάποια στατιστική δοκιμασία. Δηλαδή εκφράζεται μία **μηδενική υπόθεση (*null hypothesis*)**, που συνήθως είναι η απουσία διαφοροποίησης μεταξύ των ομάδων, και τη χρήση κάποια **τιμή δοκιμής (*test statistic*)** για να καθοριστεί το εάν η παρατηρούμενη κατανομή των δεδομένων είναι αληθοφανής υπό τη μηδενική υπόθεση. Όταν η τιμή της δοκιμής υπερβαίνων κάποιο κατώφλι η μηδενική υπόθεση απορρίπτεται.

Δυστυχώς, είναι γενικά δύσκολο να γίνουν στατιστικές υποθέσεις σωστά και τα πορίσματα τους εύκολο παρανοούνται. Ο καθορισμός στατιστικής δοκιμασίας εμπλέκει διάφορες υποκειμενικές επιλογές, όπως:

* η στατιστική δοκιμή προς χρήση
* η μηδενική υπόθεση
* η στάθμη εμπιστοσύνης

από τον χρήστη, που σπάνια τεκμηριώνονται από το εκάστοτε πρόβλημα, αλλά βασίζονται περισσότερο σε αυθαίρετες επιλογές και στατιστική παράδωση (*Johnson, 1999*). Οι αποδείξεις που παρέχουν στον χρήστη είναι έμμεσες, ατελείς και τυπικά υπερεκτιμούν τις αποδείξης κατά της μηδενικής υποθέσεως (*Goodman, 1999*).

Μια πιο πληροφοριακή και αποτελεσματική προσέγγιση για συγκρίσεις ομάδων βασίζεται σε **εκτίμηση** έναντι **δοκιμής** και ωθείται από Μπεϋζιανή αντί frequentist πιθανότητα. Αντί να δοκιμάζεται το κατά πόσον δύο (ή περισσότερες) ομάδες διαφέρουν, **εκτιμούμε** τη διαφορά τους, κάτι που συνοδεύεται από εκτίμηση αβεβαιώτητας **επιστημική αβεβαιότητα (*epistemic uncertainty*)** και αβεβαιότητα λόγο έμφυτης στοχαστικότητας του συστήματος **κυβευτική αβεβαιότητα (*aleatory uncertainty*)**.

Θα εξαιτάσουμε αρχικά την απλούστερη περίπτωση, δύο ομάδων που αφορά τη μεταβλητή "προσθήκη νερού". Το πρώτο βήμα στη Μπεϋζιανή συμπερασματολογία είναι ο ορισμός του πλήρους μοντέλου πιθανοτητων επί του προβλήματος. Επειδή έχουμε 63 χημικές παραμέτρους ενδιαφέροντως θα επιλέξουμε πολυδιάστατη κατανομή. Επιλέγουμε την κατανομή Τ, γιατί είναι περισσότερο ανεκτική σε "ακραίες" τιμές. Η κατανομή Τ σε μία διάσταση καθορίζεται με τρείς παραμέτρους $\mathcal{T(\mu, \lambda,\nu)}$, ως εξής
$$
f(x|\mu,\lambda,\nu)\ =\ \dfrac{\Gamma (\dfrac{\nu+1}{2})}{\Gamma(\dfrac\nu2)}\Bigg(\dfrac{\lambda}{\pi\nu}\Bigg)\Bigg[1+\dfrac{\lambda(x-\mu)^2}\nu\Bigg]^{^{-\dfrac{\nu+1}{2}}}
$$

Η παράμετρος $\mu$ αντιπροσωπεύει τον μέσο όρο, η παράμετρος $\lambda$ την ακρίβεια ή αντίστοφη διακύμανση και η παράμετρος $\nu$ τους βαθμούς ελευθερίας. Οι βαθμοί ελευθερίας αντιπροσωπεύουν την "κανονικότητα" της κατανομής $(\nu\rightarrow +\infty)[\mathcal{T}\rightarrow\mathcal{N}]$ και κατανομή τείνει στην κανονική, ενώ τιμές κοντά στο μηδέν δείνουν χοντρές ουρές.

Ορίζουμε την πιθανοφάνεια του μοντέλου ως εξής:
$$
\begin{array}{c}
\mathbf{x}^{\text{Group 1}}_i\thicksim\ \mathcal{T(\nu,\mu_1,\sigma_1)}\\
\mathbf{x}^{\text{Group 2}}_i\thicksim\ \mathcal{T(\nu,\mu_2,\sigma_2)}\\
\mathbf{x}^{\text{Group 3}}_i\thicksim\ \mathcal{T(\nu,\mu_3,\sigma_3)}\\
\vdots\\
\mathbf{x}^{\text{Group G}}_i\thicksim\ \mathcal{T(\nu,\mu_G,\sigma_G)}\\
\end{array}
$$

Χάρην απλότητας υποτέθηκαν ίδιοι βαθμοί ελευθερίας αν και η υπόθεση δεν είναι απαραίτητη. Πρέπει να θέσουμε εκ των προτέρων κατανομές για τις άγνωστες παραμέτρους $\mathbf{\theta}=(\mu_1,\sigma_1,\dots,\mu_G,\sigma_G)$. Η επιλογή μας αυτή καθορίζεται από την όποια εκ των προτέρων γνώση μας. Για παράδειγμα, εάν γνωρίζουμε οτι η προσθήκη νερού αυξάνει το ελαϊκό οξύ, από βιβλιογραφική ανασκόπηση, προηγούμενες έρευνες κλπ, μπορούν να ενσωματώσουμε αυτή τη πληροφορία στο μοντέλο μας επιλέγοντας κατάλληλες κατανομές πχ
$$
\begin{array}{c}
\mu_0\thicksim\mathcal{N(\alpha,2\sigma)}\\
\mu_1\thicksim\mathcal{N(10\alpha,2\sigma)}\\
\alpha\in\mathbb{R}^+
\end{array}
$$

Δεν έχουμε κάποια τέτοια ιδιαίτερη γνώση, οπότε θα υποθέσουμε κανονική κατανομή μέσου, τον συνενομένο (*pooled*) μέσο των δεδομένων. Ομοίως και για την τυπική απόκλιση.
$$
\mu_G\thicksim\mathcal{N(\bar{x},2\sigma)}
$$

In [ ]:
means=pd.DataFrame(Xraw.mean())
means.columns=["$\mu$"]
stds=pd.DataFrame(Xraw.std())
stds.columns=["$\sigma$"]
gstats=pd.concat([means,stds],axis=1)
gstats

Παρατηρούμε κατα περίπτωση τεράστειες τιμές διακύμασης. Είναι πιθανό να οδηγηθούμε σε εξαιρετικά δύσκολη εκ των υστέρων κατανομή με αυτές τις τιμές. Ίσως να έχει νόημα να εξαιταστούν ορισμένες χωριστά. Θέτουμε ομοιόμορφες εκ των προτέρων για τις διακυμάνσεις:
$$
\sigma_G\thicksim\mathcal{U(0.5,1000)}
$$
Αρχικά ας εξαιτάσουμε την απλή περίπτωση, διαφοροποίησης του λινολεϊκού οξέως ως προς τη προσθήκη νερού

In [ ]:
# Prepping data
yeswater = Y.query("watadd_b == 1.0")
nowater = Y.query("watadd_b == .0")
X_water=Xraw.loc[Xraw.index.intersection(yeswater.index),["chemical.f_acids.lino"]]
X_no_water=Xraw.loc[Xraw.index.intersection(nowater.index),["chemical.f_acids.lino"]]
m1,m2 = X_water.mean().values[0], X_no_water.mean().values[0]
sd1, sd2 = X_water.std(ddof=1).values[0], X_no_water.std(ddof=1).values[0]

In [ ]:
with pymc.Model() as simple_linoleic_model:
    μ1 = pymc.Normal('μ1', mu = m1, sigma=2*sd1)
    μ2 = pymc.Normal('μ2', mu=m2, sigma=2*sd2)
    δμ = pymc.Deterministic('δμ', μ1-μ2)
    σ1 = pymc.Uniform('σ1', lower=1,upper=10 )
    σ2 = pymc.Uniform('σ2', lower=1,upper=10 )
    ν = pymc.Uniform('ν', lower=10, upper=100)
    y1 = pymc.StudentT('y1', mu=μ1, nu=ν , sigma=σ1, observed = X_water.values)
    y2 = pymc.StudentT('y2', mu=μ2, nu=ν , sigma=σ2, observed=X_no_water.values)
    idata = pymc.sample(1000, tune = 2000, chains=2, cores=2)

Γενικά, θα χρειαστεί να δουλέψουμε με πολλαπλούς δείκτες, συνεπώς θα χρειαστούμε πολυδιάστατες κατανομές, διαφορετικά θα πρέπει να προσαρμώσουμε ξεχωριστές μονοδιάστατες κατανομές για κάθε συνιστώσα, αγνοώντας τις μεταξύ τους σχέσεις. Οι πολυδιάσταστες κατανομές, αντικαθιστούν τη διακύμανση με **πίνακα διακύμανσης**. Για παράδειγμα, για την κανονική κατανομή:
$$
\mathcal{N}(\mu,\sigma)\ \ \text{Σε μία διάσταση}\\
\mathcal{N^n}\mathbf{\big(\mu, \Sigma\big)}\ \ \ \text{Σε n διαστάσεις χρειαζόμαστε $n\times n\ $ πίνακα συνδιακυμάνσεων ($\Sigma$)}
$$
Συνεπώς χρειαζόμαστε μέθοδο για να θέσουμε εκ των προτέρων κατανομές σε πίνακες διακυμάνσεων. Μια τέτοια ειδική κατανομή είναι η Wishart, η οποία όμως έχει ορισμένα προβληματικά χαρακτηριστικά. Μία καλύτερη προσέγγιση είναι η κατανομή **LKJCholesky**, κατανομή επί πινάκων συσχετίσεων:
$$
\mathbf{\Sigma}\ \thicksim \mathcal{LKJCholesky}(n, \eta, s)\\
\text{Έχει τρεις ελεύθερες υπερ-παραμέτρους:}\\
n\ \mathrel{\vcenter{:}}=\ \text{Οι διαστάσεις των $n\times\ n$ πινάκων συσχέτισης}\\
\eta\ \mathrel{\vcenter{:}}=\ \text{Παράμετρος σχέματος της κατανομής επί των πινάκων. Για $\eta=1$ η κατανομή είναι ομοιόμορφη ενώ, $\eta\rightarrow +\infty$ προτιμούνται πίνακες με ασθενέστες συσχετίσεις στα στοιχεία εκτός διαγωνίου}\\
s\ \mathrel{\vcenter{:}}=\ \text{Οι κατανομή των τυπικών αποκλίσεων των πινάκων συνδιακύμανσης}
$$

In [ ]:
def LKJ_show(eta, sd_dist, n):
    with pymc.Model() as test_model:
        sds = pymc.Normal.dist(sd_dist,1)
        chol,corr,stds = pymc.LKJCholeskyCov("packed_L", n=n, eta=eta, sd_dist=sds, compute_corr=True)
        # L = pymc.expand_packed_triangular(2, chol)
        Σ = pymc.Deterministic('Σ', chol.dot(chol.T))
        samples = pymc.sample_prior_predictive(samples=1)
    return samples

In [ ]:
samples = LKJ_show(1, 2.0, 100)
fig, axs = plt.subplots(ncols=3, nrows=3, figsize=(25,10))
sns.heatmap(samples.prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[0,0])
axs[0,0].set_title("$eta=1,\ s\thicksim\mathcal{HN}(2.0)$")
sns.heatmap(LKJ_show(20, 2.0, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[0,1])
axs[0,1].set_title("$eta=20,\ s\thicksim\mathcal{HN}(2.0)$")
sns.heatmap(LKJ_show(100, 2.0, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[0,2])
axs[0,2].set_title("$eta=100,\ s\thicksim\mathcal{HN}(2.0)$")
sns.heatmap(LKJ_show(1, .1, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[1,0])
axs[1,0].set_title("$eta=1,\ s\thicksim\mathcal{HN}(0.1)$")
sns.heatmap(LKJ_show(20, 0.1, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[1,1])
axs[1,1].set_title("$eta=20,\ s\thicksim\mathcal{HN}(0.1)$")
sns.heatmap(LKJ_show(100, 0.1, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[1,2])
axs[1,2].set_title("$eta=100,\ s\thicksim\mathcal{HN}(0.1)$")
sns.heatmap(LKJ_show(1, 100, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[2,0])
axs[2,0].set_title("$eta=1,\ s\thicksim\mathcal{HN}(100)$")
sns.heatmap(LKJ_show(20, 100, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[2,1])
axs[2,1].set_title("$eta=20,\ s\thicksim\mathcal{HN}(100)$")
sns.heatmap(LKJ_show(100, 100, 100).prior.stack(samples=['chain', 'draw'])['packed_L_corr'][:,:,0], cmap='viridis', ax=axs[2,2])
axs[2,2].set_title("$eta=100,\ s\thicksim\mathcal{HN}(100)$")
plt.show()

Τελικά για καταλήγουμε σε μοντέλο:
$$
\begin{array}{clc}
\mathbf{\stackrel{N\times M}{x_1} \thicksim \mathcal{T}(\mu_1, \Sigma_1, \nu_1)}&\\
\vdots &\\
\mathbf{x_K \thicksim \mathcal{T}(\mu_K, \Sigma_K, \nu_K)}&\\
&\mathbf{\nu_1=\dots=\nu_K=\nu}&\\
&\mathbf{\nu\thicksim }&\\
\end{array}
$$


In [ ]:
production_no_colmeth = pd.concat([production_targets.iloc[:,:6],production_targets.iloc[:,12:] ], axis=1)
# Drop all nans and discard the first column as it tracks nan. Discard centr column as it's been modeled previously
production_no_colmeth_no_nan = production_no_colmeth[~production_no_colmeth.isna().any(axis=1)].iloc[:,1:].drop(['centr'], axis=1)
production_no_colmeth_no_nan
# Scale inputs
X = tidy_multiindex(chem_indicators)
Y = production_no_colmeth_no_nan
# Select all samples with valid entries for both predictors and targets
valid = Y.index.intersection(X.index)
X, Y= X.loc[valid,:], Y.loc[valid,:]
# Rearrange Y columns to group variables by problem type
Y = pd.concat(
    [Y.iloc[:, :4], Y.iloc[:,6:], Y.iloc[:,[4]], Y.iloc[:,[5]]
    ],axis=1)
# Illicit values recorded as 40-50. Replaced with their mean
Y = Y.replace(dict(kbtemp={'40-50':45.0})).astype(float)
# Keep this for later
Xraw=X
X = pd.DataFrame(data=sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1)).fit_transform(X),
                columns=X.columns, index=X.index)

In [ ]:
# Prepping data
Xraw = std_scale(Xraw)
yeswater = Y.query("watadd_b == 1.0")
nowater = Y.query("watadd_b == .0")
X_water=Xraw.loc[Xraw.index.intersection(yeswater.index),:]
X_no_water=Xraw.loc[Xraw.index.intersection(nowater.index),:]
m1,m2 = X_water.mean(), X_no_water.mean()
sd1, sd2 = X_water.std(ddof=1), X_no_water.std(ddof=1)

In [ ]:
coords = {'vars':X_water.columns.tolist() }
with pymc.Model(coords=coords) as water_addition_model:
    μ_water = pymc.MvNormal('μ_water', mu = X_water.mean().values, cov = 1.5*X_water.cov().values, dims='vars')
    μ_no_water = pymc.MvNormal('μ_no_water', X_no_water.mean().values, cov = 1.5*X_no_water.cov().values, dims='vars')
    δμ = pymc.Deterministic('δμ', μ_water-μ_no_water, dims='vars')
    ν_water= pymc.HalfCauchy('ν_water', 2.0)
    ν_no_water = 1.0
    η_water = 1
    η_no_water = 1
    sd_water = pymc.HalfCauchy.dist(2)
    sd_no_water = pymc.HalfCauchy.dist(2)
    L_water, corr_water, deviations_water =  pymc.LKJCholeskyCov('LKJ_water', n=X_water.shape[1], sd_dist=sd_water,eta=η_water, compute_corr=True)
    L_no_water, corr_no_water, deviations_no_water = pymc.LKJCholeskyCov('LKJ_no_water', n=X_water.shape[1], sd_dist=sd_no_water, eta=η_no_water, compute_corr=True)
    Σ_water, Σ_no_water = pymc.Deterministic("Σ_water", L_water.dot(L_water.T)), pymc.Deterministic("Σ_no_water", L_no_water.dot(L_no_water.T))
    observed_water = pymc.MvStudentT('observed_water', mu=μ_water,nu=ν_water, scale=Σ_water, observed = X_water.values)
    observed_no_water = pymc.MvStudentT('observed_no_water', mu=μ_no_water,nu=ν_water, scale=Σ_no_water, observed = X_no_water.values)
    

In [ ]:
with water_addition_model:
    prior_samples = pymc.sample_prior_predictive(10)

In [ ]:
# Prepping data
yeswater = Y.query("watadd_b == 1.0")
nowater = Y.query("watadd_b == .0")
X_water=Xraw.loc[Xraw.index.intersection(yeswater.index),:]
X_no_water=Xraw.loc[Xraw.index.intersection(nowater.index),:]
m1,m2 = X_water.mean(), X_no_water.mean()
sd1, sd2 = X_water.std(ddof=1), X_no_water.std(ddof=1)

class EstimateGroupDifference:
    '''
        Perform Bayesian estimation  of the differences between
        groups. WIP
    '''
    def __init__(self, X:pd.DataFrame, Y:pd.DataFrame,
                 target_indicators:list[str]):
        self.X = X
        self.Y = Y
        self.target_indicators = target_indicators
        self.models = {k:None for k in target_indicators}
        self.coords = {'indicator_variables':X.columns.tolist()}
        
    def select_subsets(self):
        
        for model in self.models.keys():
            groups = self.Y.loc[:,model].unique()
            self.models[model] = {
                'ngroups':len(groups)
            }
            t = {}
            for group in groups:
                _y= self.Y.query("{varname} == {group}".format(varname=model,
                    group=group))
                t[f'Y == {group}'] = {
                        'Y_mask':_y.index,
                        'X_mask':self.X.index.intersection(_y.index)
                    }
            self.models[model] = self.models[model] | t
        return self.models
    
#     def build_models(self):
        
#         for model, model_specs in self.models.items():
           
#             with pymc.Model(coords=self.coords):
#                 μ_water = pymc.MvNormal('μ_water', mu = X_water.mean().values, cov = 2*X_water.cov().values, dims='vars')
#                 μ_no_water = pymc.MvNormal('μ_no_water', X_no_water.mean().values, cov = 2*X_no_water.cov().values, dims='vars')
#                 δμ = pymc.Deterministic('δμ', μ_water-μ_no_water, dims='vars')
#                 # ν_water = pymc.HalfCauchy('ν_water', 3.0)
#                 # ν_no_water = pymc.HalfCauchy('ν_no_water',3.0)
#                 ν_water= 1.0
#                 ν_no_water = 1.0
#                 η_water = 1.0
#                 η_no_water = 1.0
#                 sd_water = pymc.HalfCauchy.dist(5.0,shape=63)
#                 sd_no_water = pymc.HalfCauchy.dist(5.0)
#                 Lpacked1,cor1,std1 =  pymc.LKJCholeskyCov('LKJ1', n=X_water.shape[1], sd_dist=sd_no_water,eta=η_water, compute_corr=True)
#                 Lpacked2,cor2,std2 = pymc.LKJCholeskyCov('LKJ2', n=X_water.shape[1], sd_dist=sd_no_water, eta=η_no_water, compute_corr=True)
#                 L_water = pymc.Deterministic('L_water', Lpacked1.dot(Lpacked1.T))
#                 L_no_water = pymc.Deterministic('L_no_water', Lpacked2.dot(Lpacked2.T))
#                 observed_water = pymc.MvStudentT('observed_water', mu=μ_water,nu=ν_water, cov=L_water, observed = X_water.values)
#                 observed_no_water = pymc.MvStudentT('observed_no_water', mu=μ_no_water,nu=ν_no_water, cov=L_no_water, observed = X_no_water.values)


In [ ]:
prior = prior_samples.prior.stack(samples=["chain", "draw"])
sns.heatmap(prior["LKJ_water_corr"][:,:,0].values, cmap='viridis')
plt.show()

In [ ]:
with water_addition_model:
    idata = pymc.sample(draws=1000, tune=1000, chains=3, cores=3)

In [ ]:
idata.to_netcdf("water_addition_difference_estimation_model.nc")
# az.from_netcdf("water_addition_difference_estimation_model.nc")

In [ ]:
Y_mat=Y.loc[:, ["mat_p", "mat_pa", "mat_pi", "mat_m"]]
varlist = Y.columns.tolist()
# Build as many models as there are target
# variables
modeldict={k:None for k in varlist}
for target_var in Y_mat.columns:
    # For each model we fetch K-groups as distinct values of
    # the target variable (a.k.a factors)
    factors = Y.loc[:,target_var].unique()
    modeldict[target_var] = dict(n_groups=len(factors), factors = factors)
    for factor in factors:
        # We split the X matrix to sub-matrices where Y has the factor
        # value
        # Samples with the factor
        y_index = Y_mat.query(f"{target_var} == {factor}").index
        # Corresponding samples in X matrix, used to sub-samble
        x_index = Xraw.index.intersection(y_index)
        factordict={str(factor):{
            'X':x_index,
            'Y':y_index,
            }
                   }
        modeldict[target_var] = modeldict[target_var]|factordict



In [ ]:
X_raw =Xraw
coords = {'vars':X.columns.tolist() }
idx1 = Y.query("mat_p == 1.0").index
idx2 = Y.query("mat_p == 0.0").index
xidx1 = X_raw.index.intersection(idx1)
xidx2 = X_raw.index.intersection(idx2)
mu1 = X_raw.loc[xidx1,:].mean() 
mu2 =X_raw.loc[xidx2,:].mean() 
sdev1 =X_raw.loc[xidx1,:].cov()
sdev2 =X_raw.loc[xidx2,:].cov()
X_raw = (X_raw-X_raw.mean())/X_raw.std(ddof=1)

In [ ]:
with pymc.Model(coords=coords) as maturation_p_model:
    μ1 = pymc.Normal('μ1', mu = 0.00, sigma= 1, size=X_raw.shape[1], dims='vars')
    μ2 = pymc.Normal('μ2', mu = 0.00, sigma= 1, size=X_raw.shape[1], dims='vars')
    δμ12 = pymc.Deterministic('δμ12', μ1-μ2, dims='vars')
    β1 = pymc.Beta('β1', alpha=5.0, beta=1.0)
    ν1 = pymc.Deterministic('ν1', 10.0*β1)
    ν2 = pymc.Deterministic('ν2', 10.0*β1)
    η1 = 10.0
    η2 = 10.0
    sd1 = pymc.Exponential.dist(1)
    sd2 = pymc.Exponential.dist(1)
    L1, corr1, deviations1 =  pymc.LKJCholeskyCov('LKJ1', n=X_raw.shape[1], sd_dist=sd1,eta=η1, compute_corr=True)
    L2, corr2, deviations2 = pymc.LKJCholeskyCov('LKJ2', n=X_raw.shape[1], sd_dist=sd2, eta=η2, compute_corr=True)
    Σ1, Σ2 = pymc.Deterministic("Σ1", L1.dot(L1.T)), pymc.Deterministic("Σ2", L2.dot(L2.T))
    observed_1 = pymc.MvStudentT('observed_1', mu=μ1,nu=ν1, scale=Σ1, observed = X_raw.loc[xidx1,:].values)
    observed_2 = pymc.MvStudentT('observed_2', mu=μ2,nu=ν2, scale=Σ2, observed = X_raw.loc[xidx2,:].values)

In [ ]:
with maturation_p_model:
    pdata = pymc.sample_prior_predictive(100)

In [ ]:
prior = pdata.prior.stack(samples=['chain', 'draw'])

In [ ]:
prior['μ1'].T.to_pandas().reset_index().iloc[:,2:]

In [ ]:
plots = ['μ1', 'μ2']
fig, axs = plt.subplots(figsize=(25,10), ncols=1, nrows=len(plots))
for i, plot in enumerate(plots):
    _x = prior[plot].T.to_pandas().reset_index().iloc[:,2:]
    _x_=_x.melt()
    axs[i].set_title(f"Prior Mean Distributions, Group {i}")
    sns.kdeplot(_x_, x="value", hue="vars",ax=axs[i],fill=True, palette="viridis",
       alpha=.5, linewidth=0, cut=0)
plt.show()

In [ ]:
with maturation_p_model:
    idata = pymc.sample(1000, tune=2000, chains=2, cores=2)

In [ ]:
idata.to_netcdf("maturation_first_attempt.nc")

In [ ]:
with pymc.Model(coords=coords) as maturation_p_model:
    μ1 = pymc.MvNormal('μ1', mu = X_raw[xidx1,:].mean().values, cov=100*X_raw[xidx1,:].cov().values, dims='vars')
    μ2 = pymc.MvNormal('μ2', mu = 0.00, sigma= 1, size=X_raw.shape[1], dims='vars')
    δμ12 = pymc.Deterministic('δμ12', μ1-μ2, dims='vars')
    β1 = pymc.Beta('β1', alpha=5.0, beta=1.0)
    ν1 = pymc.Deterministic('ν1', 10.0*β1)
    ν2 = pymc.Deterministic('ν2', 10.0*β1)
    η1 = 10.0
    η2 = 10.0
    sd1 = pymc.Exponential.dist(1)
    sd2 = pymc.Exponential.dist(1)
    L1, corr1, deviations1 =  pymc.LKJCholeskyCov('LKJ1', n=X_raw.shape[1], sd_dist=sd1,eta=η1, compute_corr=True)
    L2, corr2, deviations2 = pymc.LKJCholeskyCov('LKJ2', n=X_raw.shape[1], sd_dist=sd2, eta=η2, compute_corr=True)
    Σ1, Σ2 = pymc.Deterministic("Σ1", L1.dot(L1.T)), pymc.Deterministic("Σ2", L2.dot(L2.T))
    observed_1 = pymc.MvStudentT('observed_1', mu=μ1,nu=ν1, scale=Σ1, observed = X_raw.loc[xidx1,:].values)
    observed_2 = pymc.MvStudentT('observed_2', mu=μ2,nu=ν2, scale=Σ2, observed = X_raw.loc[xidx2,:].values)